# Viral Genome Data from NCBI
**[Work in progress]**

This notebook downloads and standardizes viral genome data from NCBI for ingestion into the Knowledge Graph.

Data source: [NCBI](https://www.ncbi.nlm.nih.gov)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
import pandas as pd
import dateutil
from pathlib import Path
from Bio import SeqIO
from Bio import Entrez
Entrez.email = "covid19@mybinder.org"

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_HOME = Path(os.getenv('NEO4J_HOME'))
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


### NCBI Data Source
[NCBI Severe acute respiratory syndrome coronavirus 2 data hub](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=Severe%20acute%20respiratory%20syndrome%20coronavirus%202,%20taxid:2697049)

In [4]:
# constants used to generate fasta url: 
# "https://www.ncbi.nlm.nih.gov/nuccore/{genbank_accession}/?report=fasta"

ncbi_nuccore_url = "https://www.ncbi.nlm.nih.gov/nuccore/"
fasta_format = "/?report=fasta"

### Download viral genome information

In [5]:
# TODO loop over a list of taxonomy ids. 

# for now SARS-CoV-2 is hardcoded.
tax_id = "2697049" # SARS-CoV-2, NC_045512
#tax_id = "694009" # SARS, NC_004718
#tax_id = "1335626" # MERS-CoV, NC_019843
#tax_id = "333387" # Bat SARS coronavirus HKU3-1
#tax_id = "147711" # Rhinovirus A
#tax_id = "147712" # Rhinovirus B
#tax_id = "186538" # Zaire ebolavirus, NC_002549

#term = f"txid{tax_id}[Organism]+refseq[filter]" # format for reference sequences only

### Download nucleotide accession numbers for the given taxonomy id

In [6]:
#term = f"txid{tax_id}[Organism]"
term = f"txid{tax_id}[Organism]+refseq[filter]"
handle = Entrez.esearch(db="nucleotide", retmax=5000, term=term, idtype="acc")
nuc_accessions = Entrez.read(handle)
handle.close()
print("Nucleotides:", nuc_accessions['Count'])

Nucleotides: 1


In [7]:
# TODO refactor function into a separate .py file
def extract_genbank_features(gb_record, feature_type, qualifier_list) :
    answers = list()
    for (index, feature) in enumerate(gb_record.features) :
        answer = dict()
        if feature.type==feature_type :
            for qualifier in qualifier_list:
                if qualifier in feature.qualifiers :
                    #print(feature, feature.qualifiers)
                    # keep NCBI "1"-based indexing
                    answer['start'] = min(feature.location) + 1
                    answer['end'] = max(feature.location) + 1
                    #There should only be one locus_tag per feature, but there
                    #are usually several db_xref entries
                    for value in feature.qualifiers[qualifier] :
                        if value in answer :
                                print("WARNING - Duplicate key %s for %s features %i and %i" \
                                % (value, feature_type, answer[value], index))
                        else :
                            answer[qualifier] = value
                else:
                    answer[qualifier] = ''
            answers.append(answer)
    return answers

In [8]:
def process_nucleotide_record(accession):
    with Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id=accession) as handle:
        record = SeqIO.read(handle, "gb")
    # TODO cache genbank files in a temporary directory, so they don't need to be downloaded from scratch
    # every time this notebook is run
    source = extract_genbank_features(record, 'source', ['db_xref','host','isolate','isolation_source','mol_type','collection_date','country'])   
    source_df = pd.DataFrame(source)
    source_df['genbank_accession'] = accession
    source_df['complete'] = record.description.endswith('complete genome')
    
    return source_df

### Concatenate all Genbank dataframes

In [9]:
nuc_ids = nuc_accessions['IdList']
#nuc_ids.append('NC_045512') # SARS-CoV-2 refseq is missing from list (why??). Add it back.
df_list = []

for nuc in nuc_ids:
    print(nuc, end=' ')
    df_list.append(process_nucleotide_record(nuc))
    
df1 = pd.concat(df_list)

NC_045512.2 

Let's work on a copy of the original data

In [10]:
df = df1.copy()
df1.head()

,start,end,db_xref,host,isolate,isolation_source,mol_type,collection_date,country,genbank_accession,complete
0,1,29903,taxon:2697049,Homo sapiens,Wuhan-Hu-1,,genomic RNA,Dec-2019,China,NC_045512.2,True


### Filter out incomplete records

In [11]:
# incomplete genomes
df.query('complete', inplace=True)

In [12]:
# filter out any records that don't match the tax id, 
# e.g., tax_ids from organisms that are lower in the taxonomy tree.
df.query(f"db_xref == 'taxon:{tax_id}'", inplace=True)

In [13]:
df.head()

,start,end,db_xref,host,isolate,isolation_source,mol_type,collection_date,country,genbank_accession,complete
0,1,29903,taxon:2697049,Homo sapiens,Wuhan-Hu-1,,genomic RNA,Dec-2019,China,NC_045512.2,True


### Create unique and interoperable identifiers

**id**: CURIE: [insdc](https://registry.identifiers.org/registry/insdc) (International Nucleotide Sequence Database Collaboration, INSDC)

**id**: CURIE: [ncbiprotein](https://registry.identifiers.org/registry/ncbiprotein) (NCBI Reference Sequences, Refseq)

**taxonomy_id**: CURIE: [taxonomy](https://registry.identifiers.org/registry/taxonomy) (NCBI Taxonomy)

A [CURIE](https://en.wikipedia.org/wiki/CURIE) (Compact URI) is a compact abbreviation for Uniform Resource Identifiers (URIs) that can be resolved by [Identifiers.org](https://identifiers.org/).

In [14]:
# remove version number from genbank accession to enable linking with other nodes
df['id'] = "insdc:" + df['genbank_accession'].apply(lambda s: s.split('.')[0])
# # NCBI reference sequences resolve through the ncbiprotein CURIE
df['id'] = df['id'].str.replace('insdc:NC_', 'ncbiprotein:NC_')

In [15]:
# parse taxonomy_id from db_xref record
df['taxonomy_id'] = 'taxonomy:' + df['db_xref'].str.split(':', expand=True)[1]
df.rename(columns={'isolate': 'name'}, inplace=True)

### Standardize data

In [16]:
# TODO: date standardization introduces artifacts, e.g. Dec 2019 -> 2019-12-01
# Add column that specifies time granularity: Y, M, D
df['collection_date'] = df['collection_date'].apply(lambda d: dateutil.parser.parse(d) if len(d) > 0 else '')

Standardize host organism and demographics

In [17]:
# TODO a find general solution to map host name to NCBI taxonomy**

# some organism specifications are ambiguous, 
# they don't match a specificn NCBI taxonomy

taxonomy_to_id = {'Human': '9606', 
                  'Homo sapiens': '9606',
                  'Rhinolophus affinis': '59477', 
                  'Mustela lutreola': '9666',
                  'Panthera tigris jacksoni': '419130',
                  'Rhinolophus sp. (bat)': '49442', # ambiguous
                  'bat': 'taxonomy:49442', # ambiguous
                  'Manis javanica': '9974',
                  'palm civet': '71116', # ambiguous
                  'Canine': '9608' # ambiguous
                 }

Split host record into host, sex, and age (e.g., Homo sapiens; female; age 40 -> Homo sapiens, female, 40)

In [18]:
# add placeholders in case the host record is empty to enbable 3-way split
#df['host'] = df['host'].apply(lambda s: ' ; ; ' if len(s) < 1 else s)
df['host'] = df['host'].apply(lambda s: s + ' ; ; ' if s.count(';') < 1 else s)
df['host'] = df['host'].str.replace('age','')
df[['host','sex','age']] = df['host'].str.split(';', 2, expand=True)
df['host'] = df['host'].str.strip()

# assign taxonomy id to host
df['host_taxonomy_id'] = 'taxonomy:' + df['host'].apply(lambda s: taxonomy_to_id.get(s, ''))
df['sex'] = df['sex'].str.strip()
df['age'] = df['age'].str.strip()
df = df.fillna('')

### List entries where taxonomy cannot be assigned

In [19]:
df.query("host_taxonomy_id == 'taxonomy:'")

,start,end,db_xref,host,name,isolation_source,mol_type,collection_date,country,genbank_accession,complete,id,taxonomy_id,sex,age,host_taxonomy_id


In [20]:
df.head()

,start,end,db_xref,host,name,isolation_source,mol_type,collection_date,country,genbank_accession,complete,id,taxonomy_id,sex,age,host_taxonomy_id
0,1,29903,taxon:2697049,Homo sapiens,Wuhan-Hu-1,,genomic RNA,2019-12-08,China,NC_045512.2,True,ncbiprotein:NC_045512,taxonomy:2697049,,,taxonomy:9606


### Assign locations
**TODO replace this section with a general geoparsing solution**

Fix inconsistencies and add missing location data

In [21]:
locations = {
    'India: Kerala State': 'India: Kerala',
    'USA: Snohomish County, WA': 'USA: WA, Snohomish County',
    'USA: San Francisco, CA': 'USA: CA, San Francisco',
    'USA: Nashville, TN': 'USA: TN, Nashville',
    'Canada: Toronto': 'Canada: Ontario, Toronto',
    'China: Shenzhen': 'China: Guangdong, Shenzhen',
    'China: Hangzhou': 'China: Zhejiang, Hangzhou',
    'China: Wuhan': 'China: Hubei, Wuhan',
    'Italy: Cagliari': 'Italy: Sardinia, Cagliari',
    'Viet Nam: Ho Chi Minh city': 'Vietnam: Ho Chi Minh City'
}
df['country'] = df['country'].apply(lambda s: locations.get(s, s))

In [22]:
states = {
        # US
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
        # Canada
        'AB': 'Alberta',
        'BC': 'British Columbia',
        'MB': 'Manitoba',
        'NB': 'New Brunswick',
        'NL': 'Newfoundland and Labrador',
        'NT': 'Northwest Territories',
        'NS': 'Nova Scotia',
        'NU': 'Nunavut',
        'ON': 'Ontario',
        'PE': 'Prince Edward Island',
        'QC': 'Quebec',
        'SK': 'Saskatchewan',
        'YT': 'Yukon'
}

Split country records into administrative areas

In [23]:
# add placeholders in case the country record is incomplete to enable 3-way split below
df['country'] = df['country'].apply(lambda s: s if ':' in s else s + ': , ')
df[['country','admin1','admin2']] = df['country'].str.split(':|,', 2, expand=True)
df['country'] = df['country'].str.strip()
df['admin2'] = df['admin2'].str.strip()
df['admin1'] = df['admin1'].str.strip()

# # expand states to full name
df['admin1'] = df['admin1'].apply(lambda s: states.get(s, s))
# create a unique ids
# TODO use unique ids from Geonames.org
df['country_id'] = df['country']
df['admin1_id'] = df['country'] + '-' + df['admin1']
df['admin2_id'] = df['country'] + '-' + df['admin1'] + '-' + df['admin2']

df.fillna(value='', inplace = True)

### Save data for Knowledge Graph Import

In [24]:
df = df[['id','name', 'taxonomy_id', 'collection_date', 'host_taxonomy_id',
         'sex', 'age', 'isolation_source']]
df.head()

,id,name,taxonomy_id,collection_date,host_taxonomy_id,sex,age,isolation_source
0,ncbiprotein:NC_045512,Wuhan-Hu-1,taxonomy:2697049,2019-12-08,taxonomy:9606,,,


In [25]:
df.to_csv(NEO4J_HOME / "import/01a-NCBIStrain.csv", index=False)